# This will download files (most likely music) from my mega import folder to mega folder in /content/, extract zip,rar,7z files, delete the archive files, convert png to jpg, convert all flac,aiff,wav files to opus, delete png,flac,aiff,wav files, archive them with each single archive being 300mb in the rar folder of /My Drive/

Don't forget to change name of the archive.

Don't forget to change directory to /content/mega/ before downloading files form mega.

Use "-p'your_password'" in 7z command if file is password protected.

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
%cd "/content/"
!curl https://rclone.org/install.sh | sudo bash
!apt install parallel imagemagick opus-tools
!wget https://www.dropbox.com/s/wuvnnfo8wg9xew8/rclone.conf https://www.dropbox.com/s/ziyqspg5hfktms9/exclude.txt

!mkdir "/root/.config/rclone/"
!mv "/content/rclone.conf" "/root/.config/rclone/"


In [0]:
%cd "/content/"
!rclone move --checkers=10 --multi-thread-cutoff=10M --multi-thread-streams=10 --retries=10 --use-mmap --buffer-size=2000M --exclude-from=/content/exclude.txt --mega-hard-delete --transfers=10 "mega:a" "/content/mega/"


In [0]:
%cd "/content/mega/"

!find -size -1k -delete
!find -iname "*.zip" -o -iname "*.rar" -o "*.7z" -print0 | parallel -0 -j 10 '7z -o{.} x {}'
!find -iname "*.zip" -o -iname "*.rar" -o "*.7z" -delete

!find -iname "*.png" -print0 | parallel -0 -j 10 'convert {} -quality 90 {.}.jpg'
!find -iname "*.flac" -o -iname "*.aiff" -o -iname "*.wav" -print0 | parallel -0 -j 10 'opusenc --quiet --discard-pictures --bitrate 192 {} {.}.opus'
!find -iname "*.flac" -o -iname "*.aiff" -o -iname "*.wav" -print0 | parallel -0 -j 10 'ffmpeg -loglevel warning -n -i {} -c:a libopus -b:a 192k {.}.opus'

!find -iname "*.flac" -o -iname "*.aiff" -o -iname "*.wav" -o "*.png" -delete


In [0]:
%cd "/content/drive/Shared drives/music/"

#Input album name
!7z -v500m -mx0 -mmt4 -pemfyr a "album_name" "/content/mega/"

%cd "/content/"


In [0]:
!rm -rf "/content/mega/"


# This download manga to "/content", then archive them into group of 500MB each file (directly in drive)
Don't forget to change the archive name.

If downloading from mangadex, always use
    
    --chapter-filter "lang == 'en'"


In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
!pip install gallery-dl


In [0]:
%cd "/content/"

#Input link
!gallery-dl "Link"

%cd "/content/drive/Shared drives/colab2/pics/manga"
!find "/content/gallery-dl/" -maxdepth 2 -mindepth 2 -type d -print0 | parallel -0 -j 10 '7z -v500m -mx0 -mmt4 -pemfyr a {}.7z {}'

%cd "/content/"
!rm -rf "/content/gallery-dl/"


                                                            OR

---



In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
!pip install manga-py


In [0]:
%cd "/content/"

#Input Link
!manga-py -t 30 -f --no-progress "Link"

%cd "/content/drive/Shared drives/colab2/pics/manga"
!find "/content/Manga/" -mindepth 1 -maxdepth 1 -type d -print0 | parallel -0 -j 10 '7z -v500m -mx0 -mmt4 -pemfyr a {}.7z {}'

%cd "/content/"
!rm -rf "/content/Manga/"


# Download torrent to wherever you want (-d)
    
-Use for torrent file    -T file.torrent

-Following only work if file is torrent (To show files)   -S


-This would well slect files to download

    --select-file=x-y,z,a-b


In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
!apt install aria2


In [0]:
#                                                      OPTIONAL


#create .torrent file from magnet link (usefull when trying to see whats inside the torrent ot to use --select-files command)
!aria2c --bt-metadata-only=true --bt-save-metadata=true -d /content -q "magnet_link"

#Show inside of torrent with index numbers
#Input torrent file
!aria2c -S '.torrent_file'


In [0]:
!aria2c --max-connection-per-server=16 --split=16 --seed-time=0 --download-result=full --auto-save-interval=20 --bt-max-open-files=10000 --save-session-interval=20 --summary-interval=10 -d "/content/Torrent" "magnet_link_or_torrent_file" 


# To download music from deezer using d-fi

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
%cd "/content/"
!pip install eyeD3
!apt install parallel
!wget https://github.com/d-fi/releases/releases/download/1.3.3/d-fi-linux.zip https://raw.githubusercontent.com/Emfyr/emfyr.github.io/master/strings_without_artist_deezer.py
!7z -od-fi x /content/d-fi-linux.zip
!chmod +x "/content/d-fi/d-fi"


In [0]:
#copy the links (albums or artists) you need and paste them below
#remember to remove newline below
%%bash
echo "paste_all_deezer_links_here" > /content/d-fi/rando.txt

echo "#!/bin/bash
" > /content/d-fi/script_to_download_albums_simultaneously.sh


In [0]:
%cd /content/d-fi
!python "/content/strings_without_artist_deezer.py"
!sed -i 's#^#/content/d-fi/d-fi -c 99 #; s/$/ \&/' "/content/d-fi/deezer.txt"
!chmod +x "/content/d-fi/script_to_download_albums_simultaneously.sh"
!cat "/content/d-fi/deezer.txt" >> "/content/d-fi/script_to_download_albums_simultaneously.sh"

!/content/d-fi/script_to_download_albums_simultaneously.sh

!find "/content/d-fi/Music" -type f ! -iname "*.*" -delete

!find . -type f -iname "*.mp3" -print0 | parallel -0 -j 30 'ffmpeg -n -i {} -an -vcodec copy {//}/cover.jpg'
!find "/content/d-fi/Music" -iname "*.mp3" -print0 | parallel -0 -j 50 'eyeD3 --remove-all-images --remove-all-lyrics {}'
!find "/content/d-fi/Music" -iname "*.mp3" -print0 | parallel -0 -j 10 'ffmpeg -n -i {} -c:a libopus -b:a 192k {.}.opus'


In [0]:
#Input archive name (below)
!7z -v500m -mx0 -mmt4 -pemfyr a "artist_name" "/content/d-fi/Music"

!find -mindepth 1 -maxdepth 1  -name "*.7z.*" -print0 | parallel -0 -j 5 'mv "{}" -t "/content/drive/Shared drives/colab2/music/"'
!rm -rf "/content/d-fi/Music"


In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
!pip install coursera-dl

!mkdir "/content/coursera/"
%cd "/content/coursera/"

#copy just the course-id from url on course on coursera site
#if needed, get new cauth from cookies.txt (using the cookie extension in firefox)
#use '--specialization' if want to download a whole specialization instead of just one course
!coursera-dl --username ayushrwt@gmail.com --password alskdjfhg1 --cauth 2fG_jfNtDy04z1jVK2zDxXOfx1XFF_pSPBiDwu5GAZG1Bo0zpoCW6E5dwIHQr9KJC1_iSwZKu_yP0wV2dOepAQ.bx-fQstQOzZLuUOBBexpUQ.4oytPJOWBFJS_lo1zIUKcvQNIuWa3OfNaudhaQdiCci8CHYxEGOzmefHNMbFao-goylXpqoGtiRkOqVdghCEC0jcs1GgIoh9Mi9DxLEG6GIw_37jEMy4GCHN2cJKApFA0iT767PcsI_tSnHwBklu-E3p9RsB1CvoLEIW9aeD5TsALAewngFDgsjGmVF_W9sb --jobs 80 --subtitle-language en --download-delay 1 --download-notebooks --aria2 --download-quizzes "course_id"

%cd "/content/drive/Shared drives/c areer/"
!find "/content/coursera/" -mindepth 1 -maxdepth 1 -type d -print0 | parallel -0 -j 10 '7z -v300m -mx0 -mmt4 -pemfyr a {}.7z {}'

!rm -rf "/content/coursera"


# Commented out IPython magic to ensure Python compatibility.
# Commented out IPython magic to ensure Python compatibility.
# %cd "/content/drive/Shared drives/colab2/Pron/Pron"
!pip install youtube-dl
!apt install parallel

%%bash
echo "Paste_all_pron_links_here" > /content/pron.txt

!cat /content/pron.txt | parallel -j 10 'youtube-dl -f 480p -ciw --no-check-certificate -o "/content/drive/Shared drives/colab2/pron/pronhub/%(title)s-%(id)s.%(ext)s" --download-archive "/content/drive/Shared drives/colab2/pron/pron_archive.txt" {} --external-downloader wget'

!youtube-upload
